# Mise en commun du code et du parser pour jeux de mots

In [45]:
import time
import requests
import shutil
from os import remove, mkdir, rmdir
from statistics import mean, geometric_mean, harmonic_mean
from os.path import exists
import pandas as pd 
from os.path import exists
from alive_progress import alive_bar

# Arborescence du sysème de fichier 
PATH            = "data/"
PATH_REQUEST    = "data/requests/"
PATH_DWH        = "data/dwh/"
PATH_DEF        = "data/def/"
PATH_E          = "data/e/"
PATH_NT         = "data/nt/"
PATH_RE         = "data/re/"
PATH_RS         = "data/rs/"
PATH_RT         = "data/rt/"

# Extension des fichiers 
EXT_REQUEST     = ".txt"
EXT_DEF         = "_def.csv"
EXT_E           = "_e.csv"
EXT_NT          = "_nt.csv"
EXT_RE          = "_re.csv"
EXT_RS          = "_rs.csv"
EXT_RT          = "_rt.csv"

# Data Warehouse
DWH_0 = "DWH_0.csv" 
DWH_1 = "DWH_1.csv" 
DWH_2 = "DWH_2.csv" 
DWH_3 = "DWH_3.csv" 
DWH_4 = "DWH_4.csv" 

class Terme: 
    def __init__(self, mot):
        self.mot = mot

        if self.isKnow(): 
            self.load()
        else: 
            self.download()

    def getMot(self): 
        return self.mot

    def getID(self): 
        return self.id

    def isKnow(self):
        return exists(PATH_REQUEST + self.mot + EXT_REQUEST)

    def request(self): 
        url = 'https://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel=' + self.mot + '&rel='
        headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
        r = requests.post(url, headers=headers)

        # Enregistrer la requetes dans un fichier text
        f = open(PATH_REQUEST + self.mot + EXT_REQUEST, "w")
        f.write(r.text)
        f.close()

        f = open(PATH_REQUEST + self.mot + EXT_REQUEST, "r")
        lines = f.readlines()
        f.close()

        requestIsGood = "<br>//&nbsp; &nbsp; &nbsp; WARNING TROP GROS.<br>TOWARD CACHE<br>\n" not in lines and "<CODE>MUTED_PLEASE_RESEND\n" not in lines and "<div class=\"jdm-warning\"><br>Le terme '" + self.mot + "' n'existe pas !</div></div>\n"

        if not requestIsGood: 
            remove(PATH_REQUEST + self.mot + EXT_REQUEST)
            

        return requestIsGood

    def download(self):
        i = 0

        while not self.request() and i < 5: 
            i += 1

        f = open(PATH_REQUEST + self.mot + EXT_REQUEST, "r")
        lines = f.readlines()
        try : 
            code = [lines.index('<CODE>\n'), lines.index('</CODE>\n') +1]
        except ValueError: 
            code = [0, 0]

        index = {}

        for i in range(code[0], code[1]):
            if lines[i] == "<def>\n":
                index["def"] = i

            if lines[i] == "// les types de noeuds (Nodes Types) : nt;ntid;'ntname'\n":
                index["nt"] = i

            if lines[i] == "// les noeuds/termes (Entries) : e;eid;'name';type;w;'formated name' \n":
                index["e"] = i

            if lines[i] == "// les types de relations (Relation Types) : rt;rtid;'trname';'trgpname';'rthelp' \n":
                index["rt"] = i

            if lines[i] == "// les relations sortantes : r;rid;node1;node2;type;w \n":
                index["rs"] = i

            if lines[i] == "// les relations entrantes : r;rid;node1;node2;type;w \n":
                index["re"] = i 

            if lines[i] == "// END\n":
                index["end"] = i 
    

        #index
        m_def   = open(PATH_DEF + self.mot + EXT_DEF, "w")
        m_nt    = open(PATH_NT  + self.mot + EXT_NT,  "w")
        m_e     = open(PATH_E   + self.mot + EXT_E,   "w")
        m_rt    = open(PATH_RT  + self.mot + EXT_RT,  "w")
        m_rs    = open(PATH_RS  + self.mot + EXT_RS,  "w")
        m_re    = open(PATH_RE  + self.mot + EXT_RE,  "w")

        # Head
        m_nt.write("nt;ntid;ntname\n")
        m_e.write("e;eid;name;type;w;help\n")
        m_rt.write("rt;rtid;trname;trgpname;rthelp\n")
        m_rs.write("r;rid;node1;node2;type;w\n")
        m_re.write("r;rid;node1;node2;type;w\n")

        try: 
            for i in range(index['def'], index['nt']):
                m_def.write(lines[i])

            for i in range(index['nt']+2, index['e']):
                m_nt.write(lines[i].replace("'", ""))

            for i in range(index['e']+2, index['rt']):
                if lines[i].count(";") <= 5:
                    m_e.write(lines[i].replace("'", ""))

            for i in range(index['rt']+2, index['rs']):
                m_rt.write(lines[i].replace(" ; ", " ").replace("'", "")) # Modifier les séparateur pour éviter les pb à l'ouverture

            if 're' in index and 'rs' in index: 
                for i in range(index['rs']+2, index['re']):
                    m_rs.write(lines[i])

                for i in range(index['re']+2, index['end']):
                    m_re.write(lines[i])
            elif 'rs' in index: 
                for i in range(index['rs']+2, index['end']):
                    m_rs.write(lines[i])
            elif 're' in index: 
                for i in range(index['re']+2, index['end']):
                    m_re.write(lines[i])
        except KeyError: 
            pass

        m_def.close()
        m_nt.close()
        m_e.close()
        m_rt.close()
        m_rs.close()
        m_re.close()

        self.load()

    def load(self):
        self.E  = pd.read_csv(PATH_E  + self.mot + EXT_E,  sep=";")
        self.NT = pd.read_csv(PATH_NT + self.mot + EXT_NT, sep=";")
        self.RE = pd.read_csv(PATH_RE + self.mot + EXT_RE, sep=";")
        self.RS = pd.read_csv(PATH_RS + self.mot + EXT_RS, sep=";")
        self.RT = pd.read_csv(PATH_RT + self.mot + EXT_RT, sep=";")
        self.R  = pd.concat([self.RS, self.RE])

        if self.RS.shape[0] > 0:
            self.id = int(self.RS['node1'][0])
        else:
            self.id = -1

        #self.id = int(self.E.loc[self.E['name'] == self.mot]['eid'][0])

    def initMyLocalDataBase(): 
        if not exists(PATH): 
            mkdir(PATH)

        if not exists(PATH_DEF): 
            mkdir(PATH_DEF)

        if not exists(PATH_REQUEST): 
            mkdir(PATH_REQUEST)

        if not exists(PATH_DWH): 
            mkdir(PATH_DWH)

            # Creer les entrepots
            DWH1 = open(PATH_DWH + DWH_1, "w")
            DWH2 = open(PATH_DWH + DWH_2, "w")
            DWH3 = open(PATH_DWH + DWH_3, "w")
            DWH4 = open(PATH_DWH + DWH_4, "w")

            # Ecrirre les entete
            DWH1.write("mot1;type_relation;name_relation;weight_relation;mot2\n")
            DWH2.write("mot1;type_relation_1;name_relation_1;weight_relation_1;mot2;type_relation_2;name_relation_2;weight_relation_2;mot3;mean;geometric_mean;harmonic_mean\n")
            DWH3.write("mot1;type_relation_1;name_relation_1;weight_relation_1;mot2;type_relation_2;name_relation_2;weight_relation_2;mot3;type_relation_3;name_relation_3;weight_relation_3;mot4;mean;geometric_mean;harmonic_mean\n")
            DWH4.write("mot1;type_relation_1;name_relation_1;weight_relation_1;mot2;type_relation_2;name_relation_2;weight_relation_2;mot3;type_relation_3;name_relation_3;weight_relation_3;mot4;type_relation_4;name_relation_4;weight_relation_4;mot5;mean;geometric_mean;harmonic_mean\n")
            
            # Fermer les fichier
            DWH1.close() 
            DWH2.close() 
            DWH3.close() 
            DWH4.close() 

        if not exists(PATH_E): 
            mkdir(PATH_E)

        if not exists(PATH_NT): 
            mkdir(PATH_NT)

        if not exists(PATH_RE): 
            mkdir(PATH_RE)

        if not exists(PATH_RS): 
            mkdir(PATH_RS)

        if not exists(PATH_RT): 
            mkdir(PATH_RT)

    def deleteMyLocalDataBase():
        if exists(PATH):
            shutil.rmtree(PATH)

    def isValidTerme(terme): 
        return " " not in terme and "<" not in terme and">" not in terme and":" not in terme and "\x9c" not in terme and "ï" not in terme and "_" not in terme

    

## Construction de l'arborescence 

In [46]:
Terme.initMyLocalDataBase() 

## Les fonctions pour extraire les relations

In [47]:
A = Terme("pigeon")
B = Terme("voler")
C = Terme("chat")
D = Terme("bite")
E = Terme("drogue")
F = Terme("sale")
G = Terme("chatte")
H = Terme("couille")
I = Terme("objet")

In [31]:
def relationDistance1(A, B, csv=False): 
    DWH1 = open(PATH_DWH + DWH_1, "a")

    relation = A.RS.loc[A.RS['node2'] == B.getID()]

    relation_A_B = {}

    with alive_bar(len(relation.index), force_tty=True) as bar:
        for i, row_i in relation.iterrows() : 
            mot1 = A.getMot()
            type_relation = row_i['type']
            name_relation = A.RT.loc[A.RT['rtid'] == row_i['type']]['trgpname'].to_string(index=False)
            weight_relation = row_i['w']
            mot2 = B.getMot() 

            relation_A_B[mot1 + " " + name_relation + " " + mot2] = weight_relation

            if csv: 
                DWH1.writelines(mot1 + ";" + str(type_relation) + ";" + name_relation + ";" + str(weight_relation) + ";" + str(mot2) + "\n")

            bar()

    DWH1.close() 

    relation_A_B_sorted = {}

    for i in sorted(relation_A_B, key=relation_A_B.get, reverse=True):
        relation_A_B_sorted[i] = relation_A_B[i]

    for r in relation_A_B_sorted: 
        print(str(r) + " : " + str(relation_A_B_sorted[r]))

    
     

In [32]:
relationDistance1(A, F, csv=True)

|████████████████████████████████████████| 1/1 [100%] in 0.0s (23.31/s)                                                 
pigeon idée associée sale : 31


In [33]:
def relationDistance2_old(A, B, mean=mean, csv=False):
    termes = []
    with alive_bar(len(A.RS['node2']), force_tty=True, title="Download ") as bar:
        for eidS in A.RS['node2']: 
            tmp = len(B.RE.loc[B.RE['node1'] == eidS])
            
            if tmp > 0: 
                terme = A.E.loc[A.E['eid'] == eidS]['name'].to_string(index=False)
                if Terme.isValidTerme(terme):
                    termes.append(terme)
            bar() 
        

    termes = list(dict.fromkeys(termes))

    relation_A_B = {}

    with alive_bar(len(termes), force_tty=True, title="Search   ") as bar:
        for terme in termes: 
            AB = Terme(terme)

            relation_A_AB = A.RS.loc[A.RS['node2'] == AB.getID()]
            for i, row_i in relation_A_AB.iterrows(): 
                relation_AB_B = B.RE.loc[B.RE['node1'] == AB.getID()]

                for j, row_j in relation_AB_B.iterrows():

                    relation_A_B[A.getMot() + " " + A.RT.loc[A.RT['rtid'] == row_i['type']]['trgpname'].to_string(index=False) + " " + AB.getMot() + " " + B.RT.loc[B.RT['rtid'] == row_j['type']]['trgpname'].to_string(index=False) + " " + B.getMot()] = row_i['w'] + row_j['w'] / 2
                    
            bar()

    relation_A_B_sorted = {}

    for i in sorted(relation_A_B, key=relation_A_B.get, reverse=True):
        relation_A_B_sorted[i] = relation_A_B[i]

    for r in relation_A_B_sorted: 
        print(str(r) + " : " + str(relation_A_B_sorted[r]))

    if csv: 
        pass

In [34]:
relationDistance2_old(G, H)

Download  |████████████████████████████████████████| 1616/1616 [100%] in 0.2s (8863.47/s)                               
Search    |████████████████████████████████████████| 17/17 [100%] in 9.1s (1.87/s)                                      
chatte idée associée sexe partie couille : 225.0
chatte glose/sens/signification sexe partie couille : 210.0
chatte idée associée sexe idée associée couille : 205.5
chatte glose/sens/signification sexe idée associée couille : 190.5
chatte caractéristique poilue caractéristique-1 couille : 146.5
chatte sentiment bonheur sentiment-1 couille : 104.0
chatte synonyme sexe partie couille : 79.0
chatte idée associée poils tout couille : 78.0
chatte idée associée poils idée associée couille : 74.0
chatte sentiment crainte sentiment-1 couille : 72.5
chatte chose>lieu femme partie couille : 70.5
chatte sentiment surprise sentiment-1 couille : 67.5
chatte sentiment satisfaction sentiment-1 couille : 66.5
chatte domaine sexe partie couille : 65.0
chatte partie

In [14]:
def relationDistance2(A, B, csv=False):
    termes = []

    with alive_bar(len(A.RS['node2']), force_tty=True, title="Download ") as bar:
        for eidS in A.RS['node2']: 
            terme = A.E.loc[A.E['eid'] == eidS]['name'].to_string(index=False)

            if Terme.isValidTerme(terme):
                termes.append(terme)
            bar() 


    termes = list(dict.fromkeys(termes))
    
    DWH2 = open(PATH_DWH + DWH_2, "a")
    relation_A_B = {}

    with alive_bar(len(termes), force_tty=True, title="Search   ") as bar:
        for terme in termes:     
            AB = Terme(terme)
            relation_A_AB = A.RS.loc[A.RS['node2'] == AB.getID()]

            for i, row_i in relation_A_AB.iterrows(): 
                relation_AB_B = AB.RS.loc[AB.RS['node2'] == B.getID()]

                for j, row_j in relation_AB_B.iterrows():
                    #mot1;type_relation_1;name_relation_1;weight_relation_1;mot2;type_relation_2;name_relation_2;weight_relation_2;mot3;mean
                    mot1 = A.getMot()
                    type_relation_1 = row_i['type']
                    name_relation_1 = A.RT.loc[A.RT['rtid'] == row_i['type']]['trgpname'].to_string(index=False)
                    name_relation_1 = name_relation_1 if name_relation_1 !=  "Series([], )" else "Null"
                    weight_relation_1 = row_i['w']
                    mot2 = AB.getMot() 
                    type_relation_2 = row_j['type']
                    name_relation_2 = AB.RT.loc[AB.RT['rtid'] == row_j['type']]['trgpname'].to_string(index=False)
                    name_relation_2 = name_relation_2 if name_relation_2 !=  "Series([], )" else "Null"
                    weight_relation_2 = row_j['w']
                    mot3 = B.getMot() 
                    mean_of_relations = mean([row_i['w'], row_j['w']])
                    mean_geo_of_relations = geometric_mean([row_i['w'] if row_i['w'] > 0 else 1, row_j['w'] if row_j['w'] > 0 else 1])
                    mean_har_of_relations = harmonic_mean([row_i['w'] if row_i['w'] > 0 else 0, row_j['w'] if row_j['w'] > 0 else 0])


                    relation_A_B[mot1 + " " + name_relation_1 + " " + mot2 + " " + name_relation_2 + " " + mot3] = mean_of_relations

                    if csv: 
                        DWH2.writelines(mot1 + ";" + 
                                        str(type_relation_1) + ";" + 
                                        name_relation_1 + ";" + 
                                        str(weight_relation_1)  + ";" + 
                                        mot2 + ";" + 
                                        str(type_relation_2) + ";" + 
                                        name_relation_2 + ";" + 
                                        str(weight_relation_2) + ";" + 
                                        mot3 + ";" + 
                                        str(mean_of_relations) + ";" + 
                                        str(mean_geo_of_relations) + ";" + 
                                        str(mean_har_of_relations) + "\n")
            bar()

    DWH2.close() 

    relation_A_B_sorted = {}

    for i in sorted(relation_A_B, key=relation_A_B.get, reverse=True):
        relation_A_B_sorted[i] = relation_A_B[i]

    for r in relation_A_B_sorted: 
        print(str(r) + " : " + str(relation_A_B_sorted[r]))


In [35]:
relationDistance2(G, H, csv=True) # csv = False implique que les données ne vont pas etre ajoutées dans l'entrepot 

Download  |████████████████████████████████████████| 1616/1616 [100%] in 0.3s (4736.60/s)                               
Search    |████████████████████████████████████████| 439/439 [100%] in 5:47.9 (1.26/s)                                  405/439 [92%] in 5:33 (1.2/s, eta: 29s) 
chatte caractéristique poilue caractéristique-1 couille : 91.5
chatte sentiment bonheur sentiment-1 couille : 52.5
chatte lieu>chose pénis idée associée couille : 47
chatte idée associée poils tout couille : 45.5
chatte lieu>chose bite synonyme couille : 45.5
chatte lieu>chose penis idée associée couille : 45.5
chatte idée associée pénis idée associée couille : 42
chatte idée associée poils idée associée couille : 41.5
chatte lieu>chose bite idée associée couille : 41.5
chatte sentiment surprise sentiment-1 couille : 40
chatte sentiment satisfaction sentiment-1 couille : 39.5
chatte sentiment crainte sentiment-1 couille : 37
chatte sentiment dégoût sentiment-1 couille : 32
chatte sentiment colère sentiment-1 

In [40]:
def relationDistance3(A, D, csv=False): 
    # A -> B -> C -> D
    sortie_A = []

    for eidS in A.RS['node2']: 
        terme_A = A.E.loc[A.E['eid'] == eidS]['name'].to_string(index=False)

        if Terme.isValidTerme(terme=terme_A):
            B = Terme(terme_A)

            for eidS in B.RS['node2']: 
                terme_B = B.E.loc[B.E['eid'] == eidS]['name'].to_string(index=False)

                if Terme.isValidTerme(terme=terme_B):
                    C = Terme(terme_B)
        
                    for eidS in C.RS['node2']: 
                        terme_C = C.E.loc[C.E['eid'] == eidS]['name'].to_string(index=False)

                        if Terme.isValidTerme(terme=terme_C):
                            print("", end="\r")
                            print(A.getMot() + " -> " + B.getMot() + " -> " + C.getMot() + " -> " + D.getMot(), end="\r")

                            if terme_C == D.getMot(): 
                                DWH3 = open(PATH_DWH + DWH_3, "a")

                                relation_A_B = A.RS.loc[A.RS['node2'] == B.getID()]
                                relation_B_C = B.RS.loc[B.RS['node2'] == C.getID()]
                                relation_C_D = C.RS.loc[C.RS['node2'] == D.getID()]

                                for i, row_i in relation_A_B.iterrows(): 
                                    for j, row_j in relation_B_C.iterrows():
                                        for k, row_k in relation_C_D.iterrows(): 
                                            mot1 = A.getMot()
                                            type_relation_1 = row_i['type']
                                            name_relation_1 = A.RT.loc[A.RT['rtid'] == row_i['type']]['trgpname'].to_string(index=False)
                                            name_relation_1 = name_relation_1 if name_relation_1 !=  "Series([], )" else "Null"
                                            weight_relation_1 = row_i['w']
                                            mot2 = B.getMot() 
                                            type_relation_2 = row_j['type']
                                            name_relation_2 = B.RT.loc[B.RT['rtid'] == row_j['type']]['trgpname'].to_string(index=False)
                                            name_relation_2 = name_relation_2 if name_relation_2 !=  "Series([], )" else "Null"
                                            weight_relation_2 = row_j['w']
                                            mot3 = C.getMot() 
                                            type_relation_3 = row_j['type']
                                            name_relation_3 = B.RT.loc[B.RT['rtid'] == row_j['type']]['trgpname'].to_string(index=False)
                                            name_relation_3 = name_relation_2 if name_relation_2 !=  "Series([], )" else "Null"
                                            weight_relation_3 = row_j['w']
                                            mot4 = D.getMot() 
                                            mean_of_relations = mean([row_i['w'], row_j['w'], row_k['w']])
                                            mean_geo_of_relations = geometric_mean([row_i['w'] if row_i['w'] > 0 else 1, row_j['w'] if row_j['w'] > 0 else 1, row_k['w'] if row_k['w'] > 0 else 1])
                                            mean_har_of_relations = harmonic_mean([row_i['w'] if row_i['w'] > 0 else 0, row_j['w'] if row_j['w'] > 0 else 0, row_k['w'] if row_k['w'] > 0 else 1])

                                            print(mot1 + " " + name_relation_1 + " " + 
                                                  mot2 + " " + name_relation_2 + " " + 
                                                  mot3 + " " + name_relation_3 + " " + 
                                                  mot4 + " : " +
                                                  str(mean_of_relations))
                                            
                                            if csv: 
                                                DWH3.writelines(mot1 + ";" + 
                                                                str(type_relation_1) + ";" + 
                                                                name_relation_1 + ";" + 
                                                                str(weight_relation_1) + ";" + 
                                                                mot2 + ";" + 
                                                                str(type_relation_2) + ";" + 
                                                                name_relation_2 + ";" + 
                                                                str(weight_relation_2) + ";" + 
                                                                mot3 + ";" + 
                                                                str(type_relation_3) + ";" + 
                                                                name_relation_3 + ";" + 
                                                                str(weight_relation_3) + ";" + 
                                                                mot4 + ";" + 
                                                                str(mean_of_relations) + ";" + 
                                                                str(mean_geo_of_relations) + ";" + 
                                                                str(mean_har_of_relations) + "\n")
                                        
                                DWH3.close() 
                                print("")


In [41]:
relationDistance3(A, B, csv=True) # csv = False implique que les données ne vont pas etre ajoutées dans l'entrepot 

pigeon idée associée oiseau idée associée effarvate idée associée voler : 122iseau -> jambe -> volerpigeon -> oiseau -> nez -> volerpigeon -> oiseau -> oeil -> volerpigeon -> oiseau -> corps -> volerpigeon -> oiseau -> ADN -> volerpigeon -> oiseau -> ADN -> volerpigeon -> oiseau -> ADN -> voler
pigeon idée associée oiseau domaine-1 effarvate domaine-1 voler : 121.33333333333333
pigeon générique oiseau idée associée effarvate idée associée voler : 77
pigeon générique oiseau domaine-1 effarvate domaine-1 voler : 76.33333333333333
pigeon glose/sens/signification oiseau idée associée effarvate idée associée voler : 44
pigeon glose/sens/signification oiseau domaine-1 effarvate domaine-1 voler : 43.333333333333336
pigeon r_aki oiseau idée associée effarvate idée associée voler : 12
pigeon r_aki oiseau domaine-1 effarvate domaine-1 voler : 11.333333333333334
pigeon domaine oiseau idée associée effarvate idée associée voler : 1
pigeon domaine oiseau domaine-1 effarvate domaine-1 voler : 0.3333

KeyboardInterrupt: 

Tu peur utiliser la méthode Terme.deleteMyLocalDataBase() pour supprimer la bdd et Terme.initMyLocalDataBase() pour construire la base

TODO :
 * Finir la méhode relationDistance4 
 * Télécharger des mots pour éviter les bugs lors de la présentation (Plus on de mots en local plus les algos sont rapides)
 * Oui on présente avec ton pc lol
 * Fait un grosse méthode qui encapsule les 4 méthodes (relationDistance1, relationDistance2, relationDistance3 et relationDistance4)

In [56]:
def relationDistance4(A, E, csv=False): 
    # A -> B -> C -> D -> E
    sortie_A = []

    for eidS in A.RS['node2']: 
        terme_A = A.E.loc[A.E['eid'] == eidS]['name'].to_string(index=False)

        if Terme.isValidTerme(terme=terme_A):
            B = Terme(terme_A)

            for eidS in B.RS['node2']: 
                terme_B = B.E.loc[B.E['eid'] == eidS]['name'].to_string(index=False)

                if Terme.isValidTerme(terme=terme_B):
                    C = Terme(terme_B)
        
                    for eidS in C.RS['node2']: 
                        terme_C = C.E.loc[C.E['eid'] == eidS]['name'].to_string(index=False)

                        if Terme.isValidTerme(terme=terme_C):
                            D = Terme(terme_C)
                
                            for eidS in D.RS['node2']: 
                                terme_D = D.E.loc[D.E['eid'] == eidS]['name'].to_string(index=False)

                                if Terme.isValidTerme(terme=terme_D):
                                    print(A.getMot() + " -> " + B.getMot() + " -> " + C.getMot() + " -> " + D.getMot(), " -> " + E.getMot(), end="\r")

                                    if terme_D == E.getMot(): 
                                        relation_A_B = A.RS.loc[A.RS['node2'] == B.getID()]
                                        relation_B_C = B.RS.loc[B.RS['node2'] == C.getID()]
                                        relation_C_D = C.RS.loc[C.RS['node2'] == D.getID()]
                                        relation_D_E = D.RS.loc[D.RS['node2'] == E.getID()]

                                        
                                        for i, row_i in relation_A_B.iterrows(): 
                                            for j, row_j in relation_B_C.iterrows():
                                                for k, row_k in relation_C_D.iterrows(): 
                                                    for l, row_l in relation_D_E.iterrows(): 
                                                        print(A.getMot() + " " + A.RT.loc[A.RT['rtid'] == row_i['type']]['trgpname'].to_string(index=False) + " " + 
                                                                        B.getMot() + " " + B.RT.loc[B.RT['rtid'] == row_j['type']]['trgpname'].to_string(index=False) + " " + 
                                                                        C.getMot() + " " + C.RT.loc[C.RT['rtid'] == row_k['type']]['trgpname'].to_string(index=False) + " "  + 
                                                                        D.getMot() + " " + D.RT.loc[D.RT['rtid'] == row_k['type']]['trgpname'].to_string(index=False) + " "  + 
                                                                        E.getMot() + " : " +
                                                                        str(mean([row_i['w'], row_j['w'], row_k['w'], row_l['w']])))

                                        print("")

# Requetes

In [42]:
df = pd.read_csv(PATH_DWH + DWH_3, sep = ";")
df 

,mot1,type_relation_1,name_relation_1,weight_relation_1,mot2,type_relation_2,name_relation_2,weight_relation_2,mot3,type_relation_3,name_relation_3,weight_relation_3,mot4,mean,geometric_mean,harmonic_mean
0,pigeon,0,idée associée,335,oiseau,0,idée associée,1,effarvate,0,idée associée,1,voler,122.000000,21.580194,2.894863
1,pigeon,0,idée associée,335,oiseau,0,idée associée,1,effarvate,0,idée associée,1,voler,122.000000,21.580194,2.894863
2,pigeon,0,idée associée,335,oiseau,27,domaine-1,-1,effarvate,27,domaine-1,-1,voler,121.333333,21.580194,0.000000
3,pigeon,6,générique,200,oiseau,0,idée associée,1,effarvate,0,idée associée,1,voler,77.000000,18.171206,2.889246
4,pigeon,6,générique,200,oiseau,27,domaine-1,-1,effarvate,27,domaine-1,-1,voler,76.333333,18.171206,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,pigeon,6,générique,200,oiseau,0,idée associée,14,roussette,0,idée associée,14,voler,81.333333,43.795191,27.331887
201,pigeon,35,glose/sens/signification,101,oiseau,0,idée associée,14,roussette,0,idée associée,14,voler,48.333333,34.875750,26.163651
202,pigeon,666,r_aki,5,oiseau,0,idée associée,14,roussette,0,idée associée,14,voler,16.333333,12.805792,9.843750
203,pigeon,3,domaine,-28,oiseau,0,idée associée,14,roussette,0,idée associée,14,voler,5.333333,7.488872,0.000000


In [44]:
df.loc[(df['mot2'] == "oiseau") & (df['mean'] > 100)]

,mot1,type_relation_1,name_relation_1,weight_relation_1,mot2,type_relation_2,name_relation_2,weight_relation_2,mot3,type_relation_3,name_relation_3,weight_relation_3,mot4,mean,geometric_mean,harmonic_mean
0,pigeon,0,idée associée,335,oiseau,0,idée associée,1,effarvate,0,idée associée,1,voler,122.000000,21.580194,2.894863
1,pigeon,0,idée associée,335,oiseau,0,idée associée,1,effarvate,0,idée associée,1,voler,122.000000,21.580194,2.894863
2,pigeon,0,idée associée,335,oiseau,27,domaine-1,-1,effarvate,27,domaine-1,-1,voler,121.333333,21.580194,0.000000
13,pigeon,0,idée associée,335,oiseau,0,idée associée,1,effarvatte,0,idée associée,1,voler,122.000000,21.580194,2.894863
14,pigeon,0,idée associée,335,oiseau,27,domaine-1,-1,effarvatte,27,domaine-1,-1,voler,121.333333,21.580194,0.000000
25,pigeon,0,idée associée,335,oiseau,0,idée associée,1,miclou,0,idée associée,1,voler,122.000000,21.580194,2.894863
26,pigeon,0,idée associée,335,oiseau,27,domaine-1,-1,miclou,27,domaine-1,-1,voler,121.333333,21.580194,0.000000
37,pigeon,0,idée associée,335,oiseau,0,idée associée,1,Condé,0,idée associée,1,voler,122.333333,21.817358,2.897870
43,pigeon,0,idée associée,335,oiseau,0,idée associée,3,Chloris,0,idée associée,3,voler,123.333333,31.800846,8.161746
49,pigeon,0,idée associée,335,oiseau,0,idée associée,4,oiseau-chat,0,idée associée,4,voler,123.333333,34.632898,10.517343


# Encapsulation

In [ ]:
def relation(A, B, deep=1, mean=mean, csv=False): 
    if deep == 0: 
        relationDistance1(A, A, csv=csv)
    elif deep == 1: 
        relationDistance1(A, B, csv=csv)
    elif deep == 2: 
        relationDistance2(A, B, csv=csv)
    elif deep == 3: 
        relationDistance3(A, B, csv=csv)
    elif deep == 4: 
        relationDistance4(A, B, csv=csv)
    else:
        print("Usage : ...")

# Delete

In [58]:
Terme.deleteMyLocalDataBase() 